In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import deepcopy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
%%time
path_file = '/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced'
name_file = 'Bars_20.12'
df = pd.read_csv(f'{path_file}/{name_file}.csv')
df = df.fillna(0)

CPU times: user 11.1 s, sys: 1.06 s, total: 12.1 s
Wall time: 14.1 s


In [4]:
# df = df.drop('Unnamed: 0',axis=1)
# df = df.rename(columns={"Mean price": "mean_price"})
# df.to_csv(f'{path_file}/{name_file}.csv', index=False)

In [5]:
df.head(2)

,datetime,mean_price,bid_0_size,bid_1_size,bid_2_size,bid_3_size,bid_4_size,bid_5_size,bid_6_size,bid_7_size,...,ask_20_size,ask_21_size,ask_22_size,ask_23_size,ask_24_size,ask_25_size,ask_26_size,ask_27_size,ask_28_size,ask_29_size
0,2020-12-01 00:00:00,19717.815,3.469,5.65,1.709,1.671,6.784,1.723,3.487,2.481,...,2.020,4.940,0.278,2.458,1.728,1.417,1.871,2.733,1.875,2.290
1,2020-12-01 00:00:01,19722.005,37.050,1.24,9.778,10.765,14.198,7.069,30.381,18.998,...,3.361,3.692,6.351,5.348,2.655,54.098,1.228,7.618,39.211,23.216


In [6]:
def calc_target_label(df_data:pd.DataFrame, col_price:str, th = 0
                     ):
    df_data['difference'] = df[f'{col_price}'].diff()
    difference = df_data['difference'].values
    target = list()
    for i in range(len(df_data)):
        if difference[i] < th:
            target.append('Down')
        elif difference[i] > th:
            target.append('Up')
        else:
            target.append('Unchange')
    return target

In [7]:
target_list = calc_target_label(df, 'mean_price', 0)
df['target'] = target_list

In [8]:
df.head(2)

,datetime,mean_price,bid_0_size,bid_1_size,bid_2_size,bid_3_size,bid_4_size,bid_5_size,bid_6_size,bid_7_size,...,ask_22_size,ask_23_size,ask_24_size,ask_25_size,ask_26_size,ask_27_size,ask_28_size,ask_29_size,difference,target
0,2020-12-01 00:00:00,19717.815,3.469,5.65,1.709,1.671,6.784,1.723,3.487,2.481,...,0.278,2.458,1.728,1.417,1.871,2.733,1.875,2.290,NaN,Unchange
1,2020-12-01 00:00:01,19722.005,37.050,1.24,9.778,10.765,14.198,7.069,30.381,18.998,...,6.351,5.348,2.655,54.098,1.228,7.618,39.211,23.216,4.19,Up


In [9]:
df[df.columns[2:62]] = df[df.columns[2:62]]*1000

In [10]:
#df = df.rename(columns={"datetime": "datetime", "mean_price": "mid price",})

In [11]:
list_bids_size = list()
list_asks_size = list()
count_lvl = 30
for i in range(count_lvl):
    list_bids_size.append(df[f'bid_{i}_size'].values)
    list_asks_size.append(df[f'ask_{i}_size'].values)

In [12]:
%%time
def calculate_imbalance(Sum_bid, Sum_ask):
    imbalance = np.log(Sum_bid)-np.log(Sum_ask)
    return imbalance

def calculate_volume(list_side, count_lvl:int):
    volume = [sum(x) for x in zip(*list_side[0:count_lvl])]
    return volume

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs


In [13]:
df_imbalance = df[['datetime','mean_price','target']]

In [14]:
%%time
list_levels = [1, 3, 5, 7, 10, 15, 20, 30]
for i in list_levels:
    df_imbalance[f'Imb_{i}'] = calculate_imbalance(calculate_volume(list_bids_size, i), 
                                                   calculate_volume(list_asks_size, i))
df_imbalance = df_imbalance.fillna(0)

CPU times: user 1min 26s, sys: 471 ms, total: 1min 26s
Wall time: 1min 26s


In [25]:
df_imbalance['target'].value_counts(normalize=True)*100
df_imbalance['Imb_3'].min(), df_imbalance['Imb_3'].max()

Unchange    54.927764
Up          22.582124
Down        22.490112
Name: target, dtype: float64

(-9.251754633759298, 8.803672978913022)

In [64]:
8.803672978913022*0.23

2.024844785149995

In [65]:
df_test = df_imbalance[df_imbalance['target']!='Unchange']
df_test['target'].value_counts(normalize=True)*100
num_lvl, l_th, r_th = '3', 3.0, 10

df_test[(df_test[f'Imb_{num_lvl}']> -r_th)&
        (df_test[f'Imb_{num_lvl}']<=-l_th)]['target'].value_counts(normalize=True)*100

df_test[(df_test[f'Imb_{num_lvl}']>=l_th)&
        (df_test[f'Imb_{num_lvl}']< r_th)]['target'].value_counts(normalize=True)*100

Up      50.102071
Down    49.897929
Name: target, dtype: float64

Down    75.274103
Up      24.725897
Name: target, dtype: float64

Up      71.635579
Down    28.364421
Name: target, dtype: float64

In [66]:
len(df_test)
len(df_test[(df_test[f'Imb_{num_lvl}']>-r_th) & (df_test[f'Imb_{num_lvl}']<=-l_th)])
len(df_test[(df_test[f'Imb_{num_lvl}']>=l_th) & (df_test[f'Imb_{num_lvl}']<  r_th)])

1153609

28274

26431

In [67]:
df_result = deepcopy(df[['datetime']])

In [68]:
def labeling(data:pd.DataFrame, nlvl:str, l_th:float, r_th:float  ):
    list_label = list()
    imbalance = data[f'Imb_{nlvl}'].values
    for i in range(len(data)):
        if -r_th < imbalance[i] <= -l_th:
            list_label.append('Down')
        elif l_th <= imbalance[i] < r_th:
            list_label.append('Up')       
        else:
            list_label.append('Unchanged')
    return list_label

In [69]:
df_result[f'Labels {num_lvl}'] = labeling(df_imbalance, num_lvl, l_th, r_th)

In [70]:
df_result.head(3)

,datetime,Labels 3
0,2020-12-01 00:00:00,Unchanged
1,2020-12-01 00:00:01,Unchanged
2,2020-12-01 00:00:02,Unchanged


In [71]:
df_result['Labels 3'].value_counts(normalize=True)*100

Unchanged    93.072073
Down          3.535502
Up            3.392425
Name: Labels 3, dtype: float64

In [72]:
df_result.to_csv('/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/Сигналы_20.12_2.csv',index=False)

In [3]:
aax = pd.read_csv('/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/BTCUSDT/AAX/0_aax_quotes__11_27-12_03.csv')

In [4]:
df_result = pd.read_csv('/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/Сигналы_20.12.csv')

In [5]:
aax.head(1)

,ask_0_price,ask_1_price,ask_2_price,ask_3_price,ask_4_price,ask_5_price,ask_6_price,ask_7_price,ask_8_price,ask_9_price,...,bid_21_size,bid_22_size,bid_23_size,bid_24_size,bid_25_size,bid_26_size,bid_27_size,bid_28_size,bid_29_size,datetime
0,17208.35,17209.68,17211.18,17211.26,17212.31,17214.76,17216.48,17218.33,17259.63,17259.64,...,86,477,11,436,7,4792,89,420,131,2020-11-27 07:07:56.599843


In [6]:
aax['mid_price'] = (aax['ask_0_price']+aax['bid_0_price'])/2
aax['diff'] = aax['mid_price'].diff()
aax = aax[['datetime','ask_0_price','ask_1_price',
           'bid_0_price','bid_1_price',
           'mid_price','diff']]

In [10]:
df_result[df_result['Labels 10'] == 'Up'][15:30]

,datetime,Labels 10
516,2020-12-01 00:08:36,Up
548,2020-12-01 00:09:08,Up
549,2020-12-01 00:09:09,Up
556,2020-12-01 00:09:16,Up
557,2020-12-01 00:09:17,Up
558,2020-12-01 00:09:18,Up
559,2020-12-01 00:09:19,Up
591,2020-12-01 00:09:51,Up
592,2020-12-01 00:09:52,Up
593,2020-12-01 00:09:53,Up


In [11]:
mark = list()
aax_diff = aax['diff'].values
for i in tqdm(range(len(aax_diff))):
    if aax_diff[i] > 0:
        mark.append('Up')
    elif aax_diff[i] < 0:
        mark.append('Down')
    else:
        mark.append('Unch')
aax['mark'] = mark

100%|██████████| 13988023/13988023 [00:19<00:00, 707683.66it/s]


In [12]:
aax.head(2)

,datetime,ask_0_price,ask_1_price,bid_0_price,bid_1_price,mid_price,diff,mark
0,2020-11-27 07:07:56.599843,17208.35,17209.68,17206.28,17204.56,17207.315,NaN,Unch
1,2020-11-27 07:07:56.602291,17209.68,17211.18,17206.28,17204.56,17207.980,0.665,Up


In [13]:
aax[(aax['datetime']>='2020-12-01 00:08:00')&
    (aax['datetime']<='2020-12-01 00:15:00')]['mark'].value_counts(normalize=True)*100

Unch    90.160071
Down     5.106550
Up       4.733379
Name: mark, dtype: float64